### Prompt Engineering Template  

Revisar si los datos se estan reemplazando corentamente en una platilla de prompt 

In [1]:
#Es básico
from langchain_core.prompts import PromptTemplate

template = "Eres un experto en marketing. Sugiere un eslogan creativo para un producto: {producto}"

prompt = PromptTemplate(
    template=template,
    input_variables=["producto"]
)

#Ver como esta compuesto el prompt
promt_completo = prompt.format(producto="equipo gps")
promt_completo

'Eres un experto en marketing. Sugiere un eslogan creativo para un producto: equipo gps'

In [2]:
#chatfromtemplate: permite definir una lista de mensajes en el template
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system","Eres un traductor del español al inglés muy preciso"),
    ("human","{texto}")
])

mensajes = chat_prompt.format_messages(texto = "Hola, ¿cómo estás?")

for m in mensajes:
    print(f"{type(m)}: {m.content}")

<class 'langchain_core.messages.system.SystemMessage'>: Eres un traductor del español al inglés muy preciso
<class 'langchain_core.messages.human.HumanMessage'>: Hola, ¿cómo estás?


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

if os.getenv("OPENAI_API_KEY"):
    print("cargado correctamente")

cargado correctamente


In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini",temperature=0.6)

In [25]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

historial = ["hola cómo estás ?","Bien","necesito de tu ayuda","cúal es tu duda"]
mensaje = "necesito comprar la mejor computadora gamer"
prompt_template = ChatPromptTemplate.from_messages([
    ("system","Eres un asistente útil y amigable llamado Chatbot pro. Responde de manera clara y concisa."),
    ("human","Historial de la conversación: \n {historial} \n\nPregunta actual: {mensaje}")
])

chain = prompt_template | llm

chain.invoke({"historial": historial, "mensaje": mensaje})

AIMessage(content='Claro, aquí tienes algunas recomendaciones para comprar una buena computadora gamer:\n\n1. **Procesador (CPU)**: Busca un procesador potente, como un Intel Core i7 o i9, o un AMD Ryzen 7 o 9.\n  \n2. **Tarjeta gráfica (GPU)**: La GPU es crucial para los juegos. Considera opciones como NVIDIA GeForce RTX 3060, 3070, 3080 o AMD Radeon RX 6000 series.\n\n3. **Memoria RAM**: Al menos 16 GB de RAM es recomendable para una experiencia fluida.\n\n4. **Almacenamiento**: Un SSD de 512 GB o más para tiempos de carga rápidos, junto con un HDD para almacenamiento adicional, es una buena combinación.\n\n5. **Placa base**: Asegúrate de que sea compatible con tu CPU y GPU.\n\n6. **Fuente de alimentación**: Debe ser de buena calidad y con suficiente potencia para soportar todos los componentes.\n\n7. **Refrigeración**: Considera un buen sistema de refrigeración para mantener la temperatura baja durante largas sesiones de juego.\n\n8. **Monitor**: No olvides un buen monitor con alta 

In [27]:
mensajes = prompt_template.format_messages(
    historial=["hola cómo estás ?", "Bien", "necesito de tu ayuda", "cuál es tu duda"],
    mensaje="necesito comprar la mejor computadora gamer"
)

for m in mensajes:
    print(f"{type(m)}: {m.content}")

<class 'langchain_core.messages.system.SystemMessage'>: Eres un asistente útil y amigable llamado Chatbot pro. Responde de manera clara y concisa.
<class 'langchain_core.messages.human.HumanMessage'>: Historial de la conversación: 
 ['hola cómo estás ?', 'Bien', 'necesito de tu ayuda', 'cuál es tu duda'] 

Pregunta actual: necesito comprar la mejor computadora gamer


#### Message PlaceHolder

Permite insertar dinámicamente una lista de mensajes en una plantilla (Template) de prompt para un modelo de lenguaje

In [29]:
from langchain_core.messages import AIMessage,HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

#crear el prompt
chat_prompt = ChatPromptTemplate.from_messages([
    ("system","Eres un aistente útil que mantien el contexto de la conversación"),
    MessagesPlaceholder(variable_name="historial"),
    ("human","{pregunta_usuario}")
])

historial_conversacion = [
    HumanMessage(content="¿Cúal es la capital de Ecuador?"),
    AIMessage(content="La capital de Ecuador es Quito"),
    HumanMessage(content="¿Cúantos habitantes tiene?"),
    AIMessage(content="Quito tiene aproximadamente 1 millon de habitantes")
]

mensajes = chat_prompt.format_messages(
    historial = historial_conversacion,
    pregunta_usuario = "Puedes identificar cual es el porcentaje de homnres"
)

In [31]:
for m in mensajes:
    print(f"{type(m)}: {m.content}")

<class 'langchain_core.messages.system.SystemMessage'>: Eres un aistente útil que mantien el contexto de la conversación
<class 'langchain_core.messages.human.HumanMessage'>: ¿Cúal es la capital de Ecuador?
<class 'langchain_core.messages.ai.AIMessage'>: La capital de Ecuador es Quito
<class 'langchain_core.messages.human.HumanMessage'>: ¿Cúantos habitantes tiene?
<class 'langchain_core.messages.ai.AIMessage'>: Quito tiene aproximadamente 1 millon de habitantes
<class 'langchain_core.messages.human.HumanMessage'>: Puedes identificar cual es el porcentaje de homnres


### Few-Shot examples

In [33]:
# Template para clasificación de sentimientos con few-shot examples
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un experto en análisis de sentimientos. Clasifica cada texto como: POSITIVO, NEGATIVO o NEUTRO."),
    MessagesPlaceholder(variable_name="ejemplos"),
    ("human", "Texto a analizar: {texto_usuario}")
])

# Few-shot examples para análisis de sentimientos
ejemplos_sentimientos = [
    HumanMessage(content="Texto a analizar: Me encanta este producto, es increíble"),
    AIMessage(content="POSITIVO"),
    HumanMessage(content="Texto a analizar: El servicio fue terrible, muy decepcionante"),
    AIMessage(content="NEGATIVO"),
    HumanMessage(content="Texto a analizar: El clima está nublado hoy"),
    AIMessage(content="NEUTRO")
]

# Generar el prompt con los ejemplos
mensajes = chat_prompt.format_messages(
    ejemplos=ejemplos_sentimientos,
    texto_usuario="¡Qué día tan maravilloso!"
)

for i, m in enumerate(mensajes):
    print(f"Mensaje {i+1} ({m.__class__.__name__}):")
    print(m.content)
    print("-" * 40)

Mensaje 1 (SystemMessage):
Eres un experto en análisis de sentimientos. Clasifica cada texto como: POSITIVO, NEGATIVO o NEUTRO.
----------------------------------------
Mensaje 2 (HumanMessage):
Texto a analizar: Me encanta este producto, es increíble
----------------------------------------
Mensaje 3 (AIMessage):
POSITIVO
----------------------------------------
Mensaje 4 (HumanMessage):
Texto a analizar: El servicio fue terrible, muy decepcionante
----------------------------------------
Mensaje 5 (AIMessage):
NEGATIVO
----------------------------------------
Mensaje 6 (HumanMessage):
Texto a analizar: El clima está nublado hoy
----------------------------------------
Mensaje 7 (AIMessage):
NEUTRO
----------------------------------------
Mensaje 8 (HumanMessage):
Texto a analizar: ¡Qué día tan maravilloso!
----------------------------------------
